In [9]:
import xarray as xr
import numpy as np
from stompy.grid import unstructured_grid
from stompy.spatial import wkb2shp
from stompy import utils
import os
import six
import matplotlib.pyplot as plt
%matplotlib notebook

In [10]:
import stompy.model.delft.waq_scenario as dwaq
from stompy.model.delft import custom_process

import local_config
six.moves.reload_module(local_config)
six.moves.reload_module(dwaq)
six.moves.reload_module(custom_process)

<module 'stompy.model.delft.custom_process' from '/home/rusty/src/stompy/stompy/model/delft/custom_process.py'>

In [11]:
#hydro = dwaq.HydroFiles(hyd_path="../v148_jun_28_2016_dwaq_merge/com-v148_jun_28_2016_dwaq_merge.hyd")
hydro = dwaq.HydroFiles(hyd_path="../v148_feb_2016_dwaq_merge/com-v148_feb_2016_dwaq_merge.hyd")
hydro.reference_originals=True # for buggy reasons, this needs to be done right away

In [12]:
g=hydro.grid()
if 0:
    g.write_cells_shp("lsb-cells.shp",[('z_bed',-hydro.element_depth())],overwrite=True)

In [13]:
class LsbTracers(local_config.LocalConfig, custom_process.CustomProcesses, dwaq.WaqModel):
    # 24 is fast, but it was creating a lot of noise.
    # 23 became nan after 5 hours.
    integration_option="\n".join([" 22.62 ; integration option",
                                  "    LOWER-ORDER-AT-BOUND NODISP-AT-BOUND",
                                  "    BALANCES-OLD-STYLE BALANCES-GPP-STYLE",
                                  "    BAL_NOLUMPPROCESSES BAL_NOLUMPLOADS BAL_NOLUMPTRANSPORT",
                                  "    BAL_NOSUPPRESSSPACE BAL_NOSUPPRESSTIME"])
    
    map_timestep=240000 # FIX! (?) Output was still 30 min
    
    def setup(self):
        self.setup_tracers()
        self.setup_monitoring()
        
    def setup_tracer_continuity(self):
        self.substances['continuity']=dwaq.Substance(initial=1.0)
        # This adds a concentration=1.0 boundary condition on all the boundaries.
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]
        self.add_bc(all_bcs,'continuity',1.0)

    def setup_tracer_san_jose(self):
        self.substances['san_jose']=dwaq.Substance(initial=0)
        self.add_bc('SAN_JOSE_flow', 'san_jose',1000.0)

    def setup_tracer_palo_alto(self):
        self.substances['palo_alto']=dwaq.Substance(initial=0)
        self.add_bc('PALO_ALTO_flow', 'palo_alto',1000.0)

    def setup_tracer_san_mateo(self):
        self.substances['san_mateo']=dwaq.Substance(initial=0)
        self.add_bc(['san_mateo'], 'san_mateo', 1000.0)

    def setup_tracer_sunnyvale(self):
        self.substances['sunnyvale']=dwaq.Substance(initial=0)
        self.add_bc('SUNNYVALE_flow', 'sunnyvale',1000.0)

    def setup_tracers(self):
        #self.setup_tracer_continuity()
        
        # Conservative tracers
        self.setup_tracer_san_jose()
        self.setup_tracer_san_mateo()
        self.setup_tracer_sunnyvale()
        self.setup_tracer_palo_alto()
        
        self.setup_pond_source()

        self.parameters['nothreads']=12

        # Decaying tracers
        
    def source_polygons(self):
        return wkb2shp.shp2geom("source_polygons.shp")
    def source_polygon(self,name):
        polys = self.source_polygons()
        idx=np.nonzero(polys['name']==name)[0]
        return polys['geom'][idx[0]]
    
    def setup_pond_source(self):
        poly = self.source_polygon("A7A8")

        # Basic diurnal cycle, growth in the pond during the day,
        # constant mortality everywhere
        #   dP/dt = in_pond(x,y) * diurnal(t) * P  - mort * P
        
        self.substances['pond_chl']=dwaq.Substance(initial=1.0)
        
        # create a spatial parameter for InPond
        self.parameters['in_pond'] = self.polygon_to_parameter(poly,0.5)
        
        # create a diurnal pattern that's about 1.0 on average
        times=np.arange( np.datetime64(hydro.time0),
                         np.datetime64(hydro.time0) + np.timedelta64(hydro.t_secs[-1],'s'),
                         np.timedelta64(1,'h'))
        daylight=utils.ideal_solar_rad(times, Imax=3.0)
        self.parameters['diurnal'] = dwaq.ParameterTemporal(times = daylight.time, values=daylight.sol_rad)
        
        # CART creates, among other options, 
        # d age_conc/dt = partial * conc
        # use conc
        # originally fed in a diurnal signal, but there was some concern that was
        # introducing too much ad hoc variability.
        # self.custom_CART(conc="in_pond",age_conc="pond_chl",partial="diurnal")
        # Drop the diurnal signal
        self.custom_CART(conc="in_pond",age_conc="pond_chl")

        # custom_Decay has been problematic, unknown reasons.
        # Use regular age tracer setup instead.
        self.substances['pond_agec']=dwaq.Substance(initial=0.0)
        self.custom_CART(conc='pond_chl',age_conc="pond_agec",conc_decay=0.05)
                                    
    def polygon_to_parameter(self,poly,value_inside=1.0,value_outside=0.0):
        g=self.hydro.grid()
        values_2d=np.where(g.select_cells_intersecting(poly),
                           value_inside,value_outside)
        
        values_3d=values_2d[self.hydro.seg_to_2d_element]
        return dwaq.ParameterSpatial(values_3d)
    
    def setup_monitoring(self):
        self.add_transects_from_shp("flux_cross_sections_LtoR.shp",add_station=True,naming='name')        
        
    def run(self,force=False):
        assert self.base_path is not None,"Must specify base_path"
        
        if not force:
            if os.path.exists(os.path.join(self.base_path,'dwaq_map.nc')):
                self.log.info("Run seems to exist -- will not run again")
                return

        self.cmd_write_hydro()
        self.cmd_write_inp()
        self.cmd_delwaq1()
        self.cmd_delwaq2()
        self.cmd_write_nc()        

# /home/rusty/src/data_lsb_tracer: make sure dwaq runs
# /home/rusty/src/data_lsb_tracer_00: add simple tracers, sections.
# ./data_lsb_tracer_01: longer, but needs nc output written.
# data_lsb_tracer_02: longer, slightly truncated end time, lower decay and use
#    the age-tracer call to get decay instead of custom_Decay. add Palo Alto source.
# data_lsb_tracer_03: winter run, no diurnal signal to pond_chl.
model = LsbTracers(hydro=hydro,base_path="./data_lsb_tracer_03",overwrite=False)

INFO:LsbTracers: start time updated from hydro: 2016-02-01T00:00:00.000000
INFO:LsbTracers: stop time update from hydro: 2016-04-01T00:00:00.000000
INFO:HydroFiles:Inferring 2D elements, 0 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 50000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 100000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 150000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 200000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 250000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 300000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 350000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 400000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 450000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 500000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 550000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D e

In [ ]:
# Short run for testing
# model.stop_time = model.start_time + np.timedelta64(60,'D') # still short...
# Full run:
model.stop_time = model.stop_time - np.timedelta64(1,'D') # avoid end-of-record issue

model.setup()

model.run(force=True)

INFO:join_features:0 open strings, 24 simple polygons
INFO:join_features:Building index
/home/rusty/src/stompy/stompy/spatial/join_features.py:514: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  p.join_id=i
/home/rusty/src/stompy/stompy/spatial/join_features.py:516: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  index=STRtree(simple_polys)
INFO:join_features:done building index
INFO:join_features:Examining largest poly left with area=3242487807.773357, 23 potential interiors
/home/rusty/src/stompy/stompy/model/delft/waq_scenario.py:1492: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  input_nodes=[g.select_nodes_nearest(p)
INFO:LsbTracers:Added 17 monitored transects from flux_cross_sections_LtoR.shp
INFO:LsbTracers:Writing hydro data


Writing paramater spatiotemporal: supporting_file is ../../v148_feb_2016_dwaq_merge/v148_feb_2016_dwaq_merge-vertdisper.seg
Writing paramater spatiotemporal: supporting_file is ../../v148_feb_2016_dwaq_merge/v148_feb_2016_dwaq_merge-tau.seg
Writing paramater spatiotemporal: supporting_file is ../../v148_feb_2016_dwaq_merge/v148_feb_2016_dwaq_merge-salinity.seg


INFO:InpFile:No dispersion arrays, will skip assignment to substances
INFO:InpFile:No velocity arrays, will skip assignment to substances
INFO:HydroFiles:Done setting boundary info
INFO:LsbTracers:Copying source process tables


Writing paramater spatiotemporal: supporting_file is ../../v148_feb_2016_dwaq_merge/v148_feb_2016_dwaq_merge-vertdisper.seg
Writing paramater spatiotemporal: supporting_file is ../../v148_feb_2016_dwaq_merge/v148_feb_2016_dwaq_merge-tau.seg
Writing paramater spatiotemporal: supporting_file is ../../v148_feb_2016_dwaq_merge/v148_feb_2016_dwaq_merge-salinity.seg
First call to waqpb_export
 Reading data......
 Writing TRM tables for LaTeX......
 Making PROCES.ASC......
 
+Process: pH_carb   
+Process: DynDepth  
+Process: DynSurf   
+Process: TotDepth  
+Process: Emersion  
+Process: Meteo     
+Process: HeatBal   
+Process: HeatBal2  
+Process: SelfCool  
+Process: MaxMacro  
+Process: Coverage  
+Process: DAYRAD    
+Process: TempMode  
+Process: Temperatur
+Process: VarSal    
+Process: Salinchlor
+Process: Veloc     
+Process: ResTim    
+Process: VertDisp  
+Process: HorzDisper
+Process: HDisperVel
+Process: HDisperAdd
+Process: Age1      
+Process: Age2      
+Process: Age3      
+P

First call to waqpb_export DONE
Calling waqpb_import


...

Return from waqpb_import
Second call to waqpb_export


Normal end


 Reading data......
 Writing TRM tables for LaTeX......
 Making PROCES.ASC......
 
+Process: CART0     
+Process: CART1     
+Process: pH_carb   
+Process: DynDepth  
+Process: DynSurf   
+Process: TotDepth  
+Process: Emersion  
+Process: Meteo     
+Process: HeatBal   
+Process: HeatBal2  
+Process: SelfCool  
+Process: MaxMacro  
+Process: Coverage  
+Process: DAYRAD    
+Process: TempMode  
+Process: Temperatur
+Process: VarSal    
+Process: Salinchlor
+Process: Veloc     
+Process: ResTim    
+Process: VertDisp  
+Process: HorzDisper
+Process: HDisperVel
+Process: HDisperAdd
+Process: Age1      
+Process: Age2      
+Process: Age3      
+Process: Age4      
+Process: Age5      
+Process: WFetch    
+Process: WDepth    
+Process: Chezy     
+Process: Wave      
+Process: CalTau    
+Process: MACDIS01  
+Process: MACDIS02  
+Process: MACDIS03  
+Process: MACDIS04  
+Process: MACDIS05  
+Process: pH_simp   
+Process: SpecCarb  
+Process: EXTINAPRO 
+Process: EXTINABVLP
+Process: EXTI

Return from second call to waqpb_export


Normal end
INFO:LsbTracers:Running delwaq1:
INFO:LsbTracers:  /opt/miniconda3/envs/dfm_t140737b/bin/delwaq1 -waq  -p /mnt/S/Data/Hydro/LSB_2023/lsb_tracers/data_lsb_tracer_03/proc_tables/proc_def
INFO:LsbTracers:delwaq1 ran in 7.93s
INFO:LsbTracers:Running delwaq2 - might take a while...
INFO:LsbTracers:  /opt/miniconda3/envs/dfm_t140737b/bin/delwaq2 waqmodel
INFO:LsbTracers:Waiting for ./data_lsb_tracer_03/waqmodel.mon to be created
INFO:LsbTracers:Okay - ./data_lsb_tracer_03/waqmodel.mon exists now
INFO:LsbTracers:0.00% Completed
INFO:LsbTracers:0.04% Completed
INFO:LsbTracers:Time remaining: 5.847h (Tue Mar 12 22:04:21 2024) 242.09x realtime
INFO:LsbTracers:0.07% Completed
INFO:LsbTracers:Time remaining: 7.245h (Tue Mar 12 23:28:25 2024) 195.30x realtime
INFO:LsbTracers:0.11% Completed
INFO:LsbTracers:Time remaining: 7.463h (Tue Mar 12 23:41:39 2024) 189.53x realtime
INFO:LsbTracers:0.14% Completed
INFO:LsbTracers:Time remaining: 7.735h (Tue Mar 12 23:58:09 2024) 182.81x realtime
IN